# Models and Capacity
Can't believe Azure has this level of access to capacity. This is something I haven't seen from all the other vendors 😉

In [1]:
from azure.identity import DefaultAzureCredential
import requests,json,os
import pandas as pd

credential = DefaultAzureCredential()
token = credential.get_token("https://management.azure.com/.default").token
headers= {'Authorization': f'Bearer {token}', 'Content-Type': 'application/json'}

In [2]:
subs = requests.get('https://management.azure.com/subscriptions?api-version=2022-12-01',headers=headers).json()
if 'value' in subs:
    subId=subs['value'][0]['subscriptionId']
    print(f"Using subscription {subId}")
else:
    print("No subscriptions found")

Using subscription cd60139f-77b6-4946-8ff6-7c2135d571fa


In [3]:
models=[]
# Get the locations
uri=f"https://management.azure.com/subscriptions/{subId}/locations?api-version=2022-12-01"
locs = requests.get(uri,headers=headers).json()
for loc in locs['value']:
    location=loc['name']
    try:
        response = requests.get(f"https://management.azure.com/subscriptions/{subId}/providers/Microsoft.CognitiveServices/locations/{location}/models?api-version=2023-05-01",headers=headers,timeout=10)
        if response.status_code == 200:
            print(f"Models {location}")
            for val in response.json()['value']:
                for sku in val['model']['skus']:
                    item={}
                    item['model']=val['model']['name']
                    item['version']=val['model']['version']
                    item['kind']=val['kind']
                    # Check if the model is in the list
                    if item not in models:
                        models.append(item)
    except Exception as ex:
        print(location,'Unable to pull',ex)           


Models eastus
Models southcentralus
Models westus2
Models westus3
Models australiaeast
Models southeastasia
Models northeurope
Models swedencentral
Models uksouth
Models westeurope
Models centralus
Models southafricanorth
Models centralindia
Models eastasia
Models japaneast
Models koreacentral
Models canadacentral
Models francecentral
Models germanywestcentral
Models italynorth
Models norwayeast
Models polandcentral
Models spaincentral
Models switzerlandnorth
Models uaenorth
Models brazilsouth
Models qatarcentral
Models global
Models eastus2
Models northcentralus
Models westus
Models japanwest
Models jioindiawest
Models westcentralus
Models southindia
Models canadaeast
Models switzerlandwest


In [4]:
def GetModelCapacities(subId,modelVersion,modelName,modelFormat):

    uri=f"https://management.azure.com/subscriptions/{subId}/providers/Microsoft.CognitiveServices/modelCapacities?api-version=2024-04-01-preview&modelFormat={modelFormat}&modelName={modelName}&modelVersion={modelVersion}"
    items=[]

    while uri:
        print(uri)
        request=requests.get(uri, headers=headers,timeout=15)
        print(request.status_code)

        
        #loop through the value attribute of the response and generate embeddings for each content
        if 200 == request.status_code:
            response=request.json()
            for val in response['value']:
                item={}
                item['id']=val['id']
                item['name']=val['name']
                item['location']=val['location']
                item['modelName']=modelName
                item['modelVersion']=modelVersion
                item['modelFormat']=modelFormat
                item['availableCapacity']=val['properties']['availableCapacity']
                items.append(item)

            #check if the odata.nextLink attribute is present in the response
            if 'nextLink' in response:
                uri=response['nextLink']
            else:
                uri=None
        else:
            print(request.content)
            uri=None
    return items


# Capacity
I am limiting to only GPT models but you can replace that and go nuts

In [ ]:
# Comment this out if you want them all
models = list(set([(m['model'],m['version'],m['kind']) for m in models if m['model'].startswith('gpt')]))

capacities=[]
for model,version,kind in models:
    print(model,version,kind)
    try:
        items = GetModelCapacities(subId,version,model,kind)
        capacities.extend(items)
    except Exception as ex:
        print(ex)

# file_path = f'Files/modelCapacities.json'
# os.makedirs(os.path.dirname(file_path), exist_ok=True)
# with open(file_path, 'w') as f:
#     json.dump(capacities, f)

# Explore
Let's take a look at where gpt-4o capacity exists

In [16]:
df = pd.DataFrame(capacities)
# Filter out the ones with no capacity
df = df[df['availableCapacity'] > 0]
df = df[df['name'] == 'ProvisionedManaged']
# Filter out the ones that are not gpt-4o
df = df[df['modelName'].str.startswith('gpt-4o')]
# List by location and model and sort by that too
df = df.sort_values(by=['location','modelName','modelVersion','modelFormat'])
# display without wrapping
pd.set_option('display.expand_frame_repr', False)
# display the location,modelName,modelVersion,modelFormat,availableCapacity
df = df[['location','modelName','modelVersion','modelFormat','availableCapacity']]
print(df)


               location modelName modelVersion modelFormat  availableCapacity
166       australiaeast    gpt-4o   2024-05-13      OpenAI                100
168         brazilsouth    gpt-4o   2024-05-13      OpenAI                100
209          canadaeast    gpt-4o   2024-05-13      OpenAI                100
179              eastus    gpt-4o   2024-05-13      OpenAI                100
183             eastus2    gpt-4o   2024-05-13      OpenAI                100
203  germanywestcentral    gpt-4o   2024-05-13      OpenAI                100
189        koreacentral    gpt-4o   2024-05-13      OpenAI                100
194      northcentralus    gpt-4o   2024-05-13      OpenAI                100
176      southcentralus    gpt-4o   2024-05-13      OpenAI                100
212          southindia    gpt-4o   2024-05-13      OpenAI                100
198       swedencentral    gpt-4o   2024-05-13      OpenAI                100
201    switzerlandnorth    gpt-4o   2024-05-13      OpenAI      